In [ ]:
from wayne_utils import load_data, save_data
import os
import sys
import pandas as pd
from tqdm import tqdm

_ROOT_PATH = "workspace/TKGT"
_OUTPUT_PATH = os.path.join( _ROOT_PATH, "test/LiveSum/v1")

test_path = os.path.join( _ROOT_PATH, "data/LiveSum/test.json")
train_path = os.path.join( _ROOT_PATH, "data/LiveSum/train.json")

test_list = load_data( test_path, "json")           # 754
train_list = load_data( train_path, "json")         # 3017

# Pure-Rule预测

In [ ]:
## Rule
from tools.live_sum_rule import main_rule
main_rule( test_list, _OUTPUT_PATH)

# Rule+llm

In [ ]:
from tools.live_sum_rag import get_prompt, post_process
from llm_inferencer import Register, Inferencer
save_path = os.path.join( _ROOT_PATH, "test/LiveSum/v2")
''''''
get_prompt( test_list, save_path )                          # 准备提示词
reg = Register()
reg.list_models()

In [ ]:
aaa = load_data( "workspace/TKGT/test/LiveSum/v2/prompt_list_only.pickle", "pickle")

In [ ]:
len(aaa[0][0])

In [ ]:
reg.add_local_model( "Qwen1.5-7B-Chat-livesum-30epoch", "workspace/TKGT/test/LiveSum/v1/models/4")

In [ ]:
kwargs = {}
prompt_list_from_path = os.path.join( save_path, "prompt_list_only.pickle")
kwargs["local_or_remote"] = "local"
kwargs["server_or_reader"] = "reader"
kwargs["model_name"] = "Qwen1.5-7B-Chat-livesum-30epoch"
kwargs["gpu_list"] = "0,1,2,3,4,5"
kwargs["local_engine"] = "vllm"
if prompt_list_from_path != None:
    kwargs["prompt_list_from_path"] = prompt_list_from_path
    kwargs["predict_list_to_path"] = prompt_list_from_path.replace( "prompt_list_only.pickle", "predict_lists_rule.pickle")
    kwargs["sample_little"] = None
inferencer = Inferencer( kwargs )

In [ ]:
# 后处理
save_path = "workspace/TKGT/test/LiveSum/v2"
test_data_path = "workspace/TKGT/data/LiveSum/test.json"
post_process( save_path, test_data_path)